In [1]:
from Train import train
from DynamicsModel import model as DynamicsModel
from env1.RealisticPendulum import RealisticPendulum as env1
import jax 
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import orbax.checkpoint as ocp
import os 


In [2]:
# Load Data:
with jnp.load('Data/data.npz') as data:
    X_data1 = data['observation_t']
    X_data2 = data['action_t']
    X_data = jnp.concatenate((X_data1, X_data2), axis=1)
    y_data = data['observation_tp']
    print(X_data.shape, y_data.shape)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)
print(X_data[:10,:], y_data[:10,:])
rng = jax.random.PRNGKey(0)

(1782000, 5) (1782000, 4)
[[-4.4447699e+00 -3.4908607e+00  5.0422489e-03 -5.5800691e+00
  -2.8572500e+00]
 [-4.4469256e+00 -3.5381429e+00 -2.0065671e-01  1.0790712e+00
   2.1163223e+00]
 [-4.4461203e+00 -3.3401802e+00 -2.1150447e-01  6.1504998e+00
   1.8793108e+00]
 [-4.4433575e+00 -3.0640976e+00  7.3925957e-02  6.7645512e+00
   8.4157705e-02]
 [-4.4393606e+00 -2.8403201e+00  5.8533812e-01  4.7761383e+00
  -1.9696319e+00]
 [-4.4334707e+00 -2.4956925e+00  7.2548842e-01  9.0264435e+00
   1.8161194e+00]
 [-4.4260988e+00 -1.9195895e+00  9.6445751e-01  1.5597723e+01
   7.4778557e-02]
 [-4.4369144e+00 -1.3830838e+00 -3.6919680e-01  1.2984116e+01
  -9.0969372e-01]
 [-4.4471965e+00 -9.1261256e-01 -2.2149411e-01  1.1362095e+01
   3.1614304e-02]
 [-4.4562159e+00 -4.9554589e-01 -2.0634162e-01  1.0109828e+01
  -9.7768736e-01]] [[-4.4469256  -3.538143   -0.20065671  1.0790712 ]
 [-4.4461203  -3.3401802  -0.21150447  6.1505    ]
 [-4.4433575  -3.0640976   0.07392596  6.764551  ]
 [-4.4393606  -2.840

In [3]:
params = train(rng, X_train, y_train, X_test, y_test, observation_size = 4,action_size = 1, learning_rate=0.001, num_epochs=100, batch_size=64)

0


TypeError: sub got incompatible shapes for broadcasting: (64, 2), (64, 4).

In [ ]:
from env1.RealisticPendulum import RealisticPendulum as env1

env1 = env1()
key = jax.random.key(0)

State = env1.reset(key)
State1 = env1.step(State, 1.0)
#print(State.pipeline_state)
#State.pipeline_state.q = jnp.array([0.0,0.0],dtype=jnp.float32)
qqd = jnp.concatenate((jnp.expand_dims(State.pipeline_state.q,axis=1),jnp.expand_dims(State.pipeline_state.qd, axis=1)), axis=0)[:,0]
qqd10 = jnp.concatenate((jnp.expand_dims(State1.pipeline_state.q,axis=1),jnp.expand_dims(State1.pipeline_state.qd, axis=1)), axis=0)

model = DynamicsModel(4,5)
input = jnp.concatenate((qqd, jnp.array([1.0])), axis=0)
qqd11 = model.apply({"params":params}, input )
print(qqd10, qqd11)



11
[[-4.4532557]
 [-3.3822134]
 [-0.7975952]
 [-1.0440478]] [-3.6959538  -2.0910776   0.08731648  4.3368535 ]


In [ ]:
ckpt_dir = os.path.abspath('./params')
checkpointer = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())
path = ocp.test_utils.erase_and_create_empty(ckpt_dir)
checkpointer.save(path / str(1), args=ocp.args.StandardSave(params))
